# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.csv













2020-03-04,"Hui Guan, Laxmikant Kishor Mokadam, Xipeng Shen, Seung-Hwan Lim, Robert Patton",FLEET: Flexible Efficient Ensemble Training for Heterogeneous Deep Neural Networks,"3rd Conference on Machine Learning and Systems (MLSys), March 2020, Austin, TX, USA. (Acceptance rate: 20% (34/170)) ",mlsys,http://guanh01.github.io/files/2020mlsys.pdf

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
publications = pd.read_csv("publications.csv", sep=",", header=0)
publications


,pub_date,authors,title,venue,url_slug,paper_url
0,2016-06-01,"Hui Guan, Wen Tang, Hamid Krim, James Keiser, ...",A topological collapse for document summarization,2016 IEEE 17th International Workshop on Signa...,spawc,http://guanh01.github.io/files/2016spawc.pdf
1,2016-12-01,"Hui Guan, Thanos Gentimis, Hamid , and James K...",First Study on Data Readiness Level,arXiv preprint arXiv:1702.02107 (Preprint),arxiv,http://guanh01.github.io/files/2017arxiv.pdf
2,2017-01-01,"Yufei Ding, Lin Ning, Hui Guan, Xipeng Shen",Generalizations of the Theory and Deployment o...,Proceedings of the 38th ACM SIGPLAN Conference...,pldi,http://guanh01.github.io/files/2017pldi.pdf
3,2017-11-14,"Hui Guan, Xipeng Shen, and Hamid Krim",Egeria: a framework for automatic synthesis of...,Proceedings of the International Conference fo...,sc,http://guanh01.github.io/files/2017sc.pdf
4,2018-02-01,"Yufei Ding, Lin Ning, Hui Guan, Xipeng Shen, M...",TOP: A Compiler-Based Framework for Optimizing...,"SysML, Feb 16th, 2018, Stanford University, 20...",top,http://guanh01.github.io/files/2018top.pdf
5,2018-09-01,"Hui Guan, Yufei Ding, Xipeng Shen, and Hamid Krim",Reuse-Centric K-Means Configuration,34th International Conference on Data Engineer...,icde,http://guanh01.github.io/files/2018icde.pdf
6,2018-11-01,"Randall Pittman, Hui Guan, Xipeng Shen, Seung-...",Exploring flexible communications for streamli...,Proceedings of the International Conference fo...,sc,http://guanh01.github.io/files/2018sc.pdf
7,2019-06-01,"Lin Ning, Hui Guan, and Xipeng Shen",Adaptive Deep Reuse: Accelerating CNN Training...,35th International Conference on Data Engineer...,icde,http://guanh01.github.io/files/2019icde.pdf
8,2019-06-11,"Hui Guan, Xipeng Shen, and Seung-Hwan Lim",Wootz: a compiler-based framework for fast CNN...,Proceedings of the 40th ACM SIGPLAN Conference...,pldi,http://guanh01.github.io/files/2019pldi.pdf
9,2019-12-09,"Hui Guan, Lin Ning, Zhen Lin, Xipeng Shen, Hui...",In-Place Zero-Space Memory Protection for CNN,Advances in Neural Information Processing Syst...,nips,http://guanh01.github.io/files/2019nips.pdf


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
import os
for row, item in publications.iterrows():
    print(item.pub_date)
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = str(item.pub_date)[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
#     if len(str(item.excerpt)) > 5:
#         md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
#     md += "\ncitation: '" + html_escape(item.citation) + "'"
    md += "\nauthors: '" + item.authors + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
#     if len(str(item.excerpt)) > 5:
#         md += "\n" + html_escape(item.excerpt) + "\n"
    
#     if len(str(item.paper_url)) > 5:
#         md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
#     md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

2016-06-01
2016-12-01
2017-01-01
2017-11-14
2018-02-01
2018-09-01
2018-11-01
2019-06-01
2019-06-11
2019-12-09
2019-12-09
2020-03-04


These files are in the publications directory, one directory below where we're working from.

In [6]:
!ls ../_publications/

2016-06-01-spawc.md 2017-11-14-sc.md    2018-11-01-sc.md    2019-12-09-fb.md
2016-12-01-arxiv.md 2018-02-01-top.md   2019-06-01-icde.md  2019-12-09-nips.md
2017-01-01-pldi.md  2018-09-01-icde.md  2019-06-11-pldi.md  2020-03-04-mlsys.md


In [7]:
!cat ../_publications/2018-09-01-icde.md

---
title: "Reuse-Centric K-Means Configuration"
collection: publications
permalink: /publication/2018-09-01-icde
date: 2018-09-01
venue: '34th International Conference on Data Engineering (ICDE), 2018. (short paper) (Acceptance rate: 23%)'
paperurl: 'http://guanh01.github.io/files/2018icde.pdf'
authors: 'Hui Guan, Yufei Ding, Xipeng Shen, and Hamid Krim'
---